<a href="https://colab.research.google.com/github/Taichi2005/GoogleColab_Whisper/blob/main/notebooks/%E3%80%90%E3%83%97%E3%83%AC%E3%82%A4%E3%83%AA%E3%82%B9%E3%83%88%E5%AF%BE%E5%BF%9C%E7%89%88%E3%80%91%E5%8B%95%E7%94%BBURL%E3%81%8B%E3%82%89%E9%AB%98%E7%B2%BE%E5%BA%A6%E6%96%87%E5%AD%97%E8%B5%B7%E3%81%93%E3%81%97%E5%AE%9F%E8%A1%8C%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%97%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### **【完成版】動画URLから高精度文字起こし実行スクリプト**

#以下の3つのセルを上から順番に実行してください。

#### **セル1: 環境構築**
#このセルでは、文字起こしに必要なライブラリやツールをすべてインストールします。初回実行時は数分かかることがあります。

# セル1: 環境構築
# -------------------------------------------------------------------------------------------
# GPUが有効になっているかを確認
print("▼ GPUの確認")
!nvidia-smi

# 必要なライブラリとツールをインストール
# yt-dlp: 様々なサイトから動画や音声をダウンロードするツール
# faster-whisper: 高速化されたWhisperモデル
# ffmpeg: 動画・音声の処理・変換に必須のツール
print("\n▼ 必要なライブラリとツールのインストール")
!pip install -U yt-dlp -q
!pip install faster-whisper==1.0.3 -q
!pip install nvidia-cublas-cu12==12.6.4.1 -q
!pip install nvidia-cudnn-cu12==9.10.2.21 -q
!apt-get update && apt-get install -y ffmpeg -qq

print("\n✅ 環境構築が完了しました。")
print("次のセルに進んでGoogle Driveに接続してください。")

▼ GPUの確認
Sun Oct 26 00:54:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   63C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+--------------------------------------

In [ ]:
#### **セル2: Google Driveへの接続 (任意)**
#文字起こししたテキストファイルをGoogle Driveに保存したい場合に、このセルを実行してGoogle Driveをマウント（接続）してください。

# -------------------------------------------------------------------------------------------
# セル2: Google Driveへの接続
# -------------------------------------------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')
print("\n✅ Google Driveへの接続が完了しました。")
print("最後のセルに進んで、文字起こしを実行してください。")

Mounted at /content/drive

✅ Google Driveへの接続が完了しました。
最後のセルに進んで、文字起こしを実行してください。


In [ ]:
#@title 🚀 URLから高精度文字起こし実行（プレイリスト対応版）
#@markdown ### 1. 動画のURLと出力先の設定
#@markdown ---
#@markdown 文字起こししたい動画のURLを入力してください。
video_url = "https://www.youtube.com/watch?v=8MF7Gn19bbI&list=PLO80H233NKGwNquoL-STz1h6X7vJK65et" #@param {type:"string"}

#@markdown 結果（.txtファイル）を保存するフォルダのパスを指定します。
output_dir = "/content/drive/MyDrive/Colab/Whisper_Transcripts/output_transcripts" #@param {type:"string"}

#@markdown **プレイリスト処理**: URLがプレイリストの場合、チェックを入れるとリスト内の全動画を処理します。チェックなし（デフォルト）の場合は、その動画単体のみ処理します。
enable_playlist = True #@param {type:"boolean"}

#@markdown ### 2. モデルとパフォーマンス設定
#@markdown ---
#@markdown **モデル**: `Zoont/...-int8-ct2`は精度を維持しつつ最速・省メモリな**総合推奨モデル**です。
model_name = "Zoont/faster-whisper-large-v3-turbo-int8-ct2" #@param ["Zoont/faster-whisper-large-v3-turbo-int8-ct2", "deepdml/faster-whisper-large-v3-turbo-ct2", "large-v3", "large-v2", "distil-large-v3", "medium", "small", "base", "tiny"]
#@markdown **計算タイプ**: `int8`モデルには`int8_float16`、`float16`モデルには`float16`の組み合わせを推奨します。
compute_type = "int8_float16" #@param ["int8_float16", "float16", "int8", "float32"]

#@markdown ### 3. VAD (音声区間検出) 設定
#@markdown ---
#@markdown VADを有効にすると、無音区間を自動で除去し、文字起こしの精度を向上させることができます。
use_vad_filter = True #@param {type:"boolean"}
vad_min_silence_duration_ms = 300 #@param {type:"slider", min:100, max:2000, step:100}

#@markdown ### 4. 高度な設定（オプション）
#@markdown ---
beam_size = 5 #@param {type:"slider", min:1, max:10, step:1}
#@markdown **中間ファイルのクリーンアップ**: 処理完了後に一時音声ファイルを削除します。
cleanup_audio_file = True #@param {type:"boolean"}


# --- ここから下は実行コード ---
import os
import yt_dlp
from faster_whisper import WhisperModel
import time
from datetime import datetime, timezone, timedelta
import torch
import glob
import shutil

def get_current_timestamp():
    JST = timezone(timedelta(hours=+9))
    return datetime.now(JST).strftime('%Y-%m-%d %H:%M:%S')

def format_filename(title, video_id):
    """ファイル名に使用できない文字を置換し、長さを制限する"""
    safe_title = "".join(c if c.isalnum() or c in (' ', '-', '_', '.') else '_' for c in title)
    return f"{safe_title[:50]}_{video_id}"

def run_pipeline():
    if not video_url:
        print("❌ エラー: 動画のURLが入力されていません。")
        return

    print(f"{get_current_timestamp()} --- 1. パイプライン開始 ---")
    os.makedirs(output_dir, exist_ok=True)

    # 作業用の一時ディレクトリをクリーンアップして再作成
    temp_audio_dir = "/content/temp_audio_processing"
    if os.path.exists(temp_audio_dir):
        shutil.rmtree(temp_audio_dir)
    os.makedirs(temp_audio_dir, exist_ok=True)

    # 2. モデルのロード
    print(f"\n{get_current_timestamp()} --- 2. モデルロード: {model_name} ---")
    try:
        model = WhisperModel(model_name, device="cuda", compute_type=compute_type)
    except Exception as e:
        print(f"❌ モデルロードエラー: {e}\n   -> ランタイムがGPUか確認してください。")
        return
    print("✅ モデルロード完了")

    # 3. ダウンロード処理
    print(f"\n{get_current_timestamp()} --- 3. 音声ダウンロード開始 ---")
    print(f"対象URL: {video_url}")
    print(f"プレイリスト処理: {'有効' if enable_playlist else '無効 (単体動画のみ)'}")

    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        # タイトルとIDをファイル名に含めることで後で識別しやすくする
        'outtmpl': os.path.join(temp_audio_dir, '%(title)s [%(id)s].%(ext)s'),
        'quiet': True,
        'no_warnings': True,
        'noplaylist': not enable_playlist, # ここでプレイリストの挙動を制御
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])
    except Exception as e:
        print(f"⚠️ ダウンロード中に一部エラーが発生しました (続行可能な場合があります): {e}")

    # 4. ダウンロードされた全ファイルを検出して順次処理
    audio_files = glob.glob(os.path.join(temp_audio_dir, "*.wav"))
    total_files = len(audio_files)

    if total_files == 0:
        print("❌ エラー: 音声ファイルがダウンロードされませんでした。URLを確認してください。")
        return

    print(f"\n{get_current_timestamp()} --- 4. 文字起こし処理開始 (対象: {total_files}件) ---")

    for i, audio_path in enumerate(audio_files, 1):
        file_name = os.path.basename(audio_path)
        # ファイル名から仮のタイトルとIDを抽出（簡易的）
        base_name = os.path.splitext(file_name)[0]
        print(f"\n[{i}/{total_files}] 処理中: {base_name}")

        start_time = time.time()
        try:
            segments, info = model.transcribe(
                audio_path,
                beam_size=beam_size,
                vad_filter=use_vad_filter,
                vad_parameters=dict(min_silence_duration_ms=vad_min_silence_duration_ms)
            )
            print(f"   -> 言語: {info.language} (確率: {info.language_probability:.1%})")

            # 結果の保存
            safe_name = format_filename(base_name, "")
            output_txt_path = os.path.join(output_dir, f"{safe_name}.txt")

            with open(output_txt_path, "w", encoding="utf-8") as f:
                f.write(f"元ファイル名: {file_name}\n")
                f.write(f"処理日時: {get_current_timestamp()}\n")
                f.write(f"モデル: {model_name} ({compute_type})\n\n---\n\n")
                for segment in segments:
                    f.write(f"[{segment.start:0>7.2f}s -> {segment.end:0>7.2f}s] {segment.text.strip()}\n")

            print(f"   ✅ 完了 ({time.time() - start_time:.1f}秒): {os.path.basename(output_txt_path)}")

        except Exception as e:
            print(f"   ❌ このファイルの処理中にエラーが発生しました: {e}")

        finally:
            if cleanup_audio_file and os.path.exists(audio_path):
                os.remove(audio_path)

    print(f"\n{get_current_timestamp()} --- 🎉 全ての処理が完了しました ---")

run_pipeline()

2025-10-26 10:25:05 --- 1. パイプライン開始 ---

2025-10-26 10:25:05 --- 2. モデルロード: Zoont/faster-whisper-large-v3-turbo-int8-ct2 ---
✅ モデルロード完了

2025-10-26 10:25:06 --- 3. 音声ダウンロード開始 ---
対象URL: https://www.youtube.com/watch?v=8MF7Gn19bbI&list=PLO80H233NKGwNquoL-STz1h6X7vJK65et
プレイリスト処理: 有効

2025-10-26 10:31:40 --- 4. 文字起こし処理開始 (対象: 15件) ---

[1/15] 処理中: プロレス合宿 Part2 voomly output 20251017 142247 [OQzVGwl1buo]
   -> 言語: ja (確率: 99.9%)
   ✅ 完了 (186.7秒): プロレス合宿 Part2 voomly output 20251017 142247 _OQzVGw_.txt

[2/15] 処理中: プロレス合宿 Part3 voomly output 20251017 162043 [v5c7s2e0q8Y]
   -> 言語: ja (確率: 99.9%)
   ✅ 完了 (231.3秒): プロレス合宿 Part3 voomly output 20251017 162043 _v5c7s2_.txt

[3/15] 処理中: プロレス研究会 in 2022 Part5 [z0DDs_PTzdQ]
   -> 言語: ja (確率: 99.9%)
   ✅ 完了 (157.1秒): プロレス研究会 in 2022 Part5 _z0DDs_PTzdQ__.txt

[4/15] 処理中: プロレス合宿 Part5 voomly output 20251017 163006 [C3FjZ5ecW9c]
   -> 言語: ja (確率: 99.8%)
   ✅ 完了 (446.0秒): プロレス合宿 Part5 voomly output 20251017 163006 _C3FjZ5_.txt

[5/15] 処理中: プロレス研究会 in 2